## El Niño / Southern Oscillation (ENSO) a partir de Temperatura Superficial do Mar (TSM)

**Author original:** [Ryan Abernathey](http://rabernat.github.io)

**Modificado por:** [Filipe Fernandes](https://github.com/ocefpaf)

De acordo com a [NOAA](https://www.esrl.noaa.gov/psd/enso/):
>  El Niño e La Niña, chamados conjuntamente de El Niño Southern Oscillation (ENSO), são desvios periódicos do esperado na TSM em regiões do Pacífico Equatorial. Essas águas do mar mais quentes ou mais frias que o normal podem afetar padrões do tempo em todo o globo influênciando sistemas de alta e baixa pressão, ventos, e precipitação. ENSO pode trazer humidade necessária em uma certa região enquanto causa  extremos de água demasiada ou escassa em outras.

Nesse notebook usaremos a biblioteca python [xarray](http://xarray.pydata.org/en/latest/) para examinar dados de TSM do produto da NOAA's [Extended Reconstructed Sea Surface Temperature (ERSST) v5](https://www.ncdc.noaa.gov/data-access/marineocean-data/extended-reconstructed-sea-surface-temperature-ersst-v5).

Demostraremos como calcular o [index do Niño 3.4](https://www.ncdc.noaa.gov/teleconnections/enso/indicators/sst/).

In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
import hvplot.xarray
import holoviews as hv
hv.extension("bokeh")

### Acessando Dados diretamente da NOAA via  OpenDAP

In [ ]:
url = "https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"
ds = xr.open_dataset(url)
ds

### Selecionar apenas as partes mais "recentes."

In [ ]:
ds = ds.sel(time=slice("1960", "2018"))
ds

### Tamanho do download

In [ ]:
ds.nbytes/1e6

# Faz o download dos dados

(Pode ser lento dependendo da conexão. Qualquer coisa reduza a série temporal mais um pouco.)

In [ ]:
ds.load()

### Plot Interativo

In [ ]:
%output holomap="scrubber" fps=3

ds.sst.hvplot("lon", "lat", cmap="Magma").redim.range(sst=(-2, 30))

### Calcula a Anomalia Mensal Climatologica

In [ ]:
sst_clim = ds.sst.groupby("time.month").mean(dim="time")

sst_anom = ds.sst.groupby("time.month") - sst_clim

### Remove tendência linear do sinal

In [ ]:
from scipy.signal import detrend


sst_anom_detrended = xr.apply_ufunc(
    detrend,
    sst_anom.fillna(0),
    kwargs={"axis": 0}).where(~sst_anom.isnull())

### Plota a média global

Para fazer a média global precisamos "pesar" os pontos usando o coseno da latitude.

In [ ]:
weights = np.cos(np.deg2rad(ds.lat)).where(~sst_anom[0].isnull())
weights /= weights.mean()

(sst_anom * weights).mean(dim=["lon", "lat"]).plot(label="raw")
(sst_anom_detrended * weights).mean(dim=["lon", "lat"]).plot(label="detrended")
plt.grid()
plt.legend();

In [ ]:
sst_anom_detrended.hvplot("lon", "lat", cmap="RdBu_r").redim.range(sst=(-2, 2))

### Calculando o Index Oceânico do Niño

In [ ]:
sst_anom_nino34 = sst_anom_detrended.sel(lat=slice(5, -5), lon=slice(360-170, 360-120))
sst_anom_nino34_mean = sst_anom_nino34.mean(dim=("lon", "lat"))
oni = sst_anom_nino34_mean.rolling(time=3, center=True).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3.75))
oni.plot(ax=ax)
ax.grid()
ax.axhline(y=0, color="black")
ax.set_ylim(-3, 3)
# ax.set_xlim("1960", "2020")
plt.ylabel("Anomaly (dec. C)");

![](https://camo.githubusercontent.com/f4fb57e583e664d9b5f1861d4b062ad0a32aeac1/68747470733a2f2f7777772e6e6364632e6e6f61612e676f762f6d6f6e69746f72696e672d636f6e74656e742f74656c65636f6e6e656374696f6e732f656c6e2d662d70672e676966)